In [15]:
import pandas as pd

# Parquetファイルのパス
parquet_file_path = '/data/satori_hdd4/takumi/steve_ssd2/matsuo_llm/llm/data_expe/CC-MAIN-2021-04-batch0-iter0.parquet'

# Parquetファイルを読み込む
df = pd.read_parquet(parquet_file_path)
print(df.head())


Empty DataFrame
Columns: []
Index: []


In [2]:
%reload_ext autoreload
%autoreload 2

#.

"""
mc4を掃除して､fasttextでgood/badを分類して､クラスタリングして記録するサンプルコード
annotationもこのnotebookで行います

"""
from datasets import load_dataset


#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

"""
#oscarなども読み込める
#ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)

"""

dataset=mc4_dataset

/data/satori_hdd4/takumi/steve_ssd2/experiment/cramming/venv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mc4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/data/satori_hdd4/takumi/.cache/huggingface/modules/datasets_modules/datasets/mc4/78f7a2b7e2524fa44ee464ef429d011c365f5fe129283869e7fd76856aacb83a/mc4.py:284: FutureWarning: Dataset 'mc4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


In [110]:
import os
import sys
sys.path.append('/data/satori_hdd4/takumi/steve_ssd2/matsuo_llm/JapaneseWarcParser/mc4s/src/cleaner')
from src.cleaner.auto_cleaner import clean_text
from src.cleaner import url_filter
from src.cleaner import rule_based_docment_checker
from src.cleaner import  hojichar_filter 
from src.cleaner import  repeated_phrase
from src.cleaner import footer
#from src.classifier.DatasetAnnotator import DatasetAnnotator

In [19]:
print(url_filter.u_filter("https://5ch./wiki/Python")) 

True


In [122]:
print(repeated_phrase.repeated_id("")) 

In [159]:
text="""これはテストの文章です。
この記事へのトラックバック一覧です。
無断転載を禁ず。000000000000000000000000
クリックしてください。000000000000"""
print(footer.remove_footer(text))

これはテストの文章です。
無断転載を禁ず。000000000000000000000000
クリックしてください。000000000000


In [ ]:
annotator=DatasetAnnotator(dataset,clean_func=clean_text,n_preload=50000)

In [ ]:
#annotations
annotator.ask_annotations()

In [ ]:
#fasttextで訓練
annotator.train_fasttext(autotuneDuration=120,wordNgrams=2)

In [ ]:
#テキストの教師なしクラスタリング: 結局使わない

#modelをannotations/text_labelsに保存しておく
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.bin.gz
#!gzip -d cc.ja.300.bin.gz

In [ ]:
from src.classifier.NounClustering import NounClustering
#clf=NounClustering()

#clf.train_wiki(n_samples=10**8) #wikipediaのタイトルで教師なしクラスタリングの学習

In [ ]:
#モデルの挙動確認
#カテゴリの分類がかなりいまいち｡
"""
for i in range(100):
    text=annotator.get_text_by_id(i)
    text_=text.replace("\n","")
    if text=="":continue
    is_noise=(annotator.predict(text))
    genre=clf.predict(text)
    print(is_noise,genre,text_[:300])
"""

In [50]:
print(rule_based_docment_checker.process_text("あカカカｷｷｷﾝああ"))

カタカナの割合：0.7



In [170]:
n_split=10**2
import os

corpus_dir="corpus/test"
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

cnt=0
record_id=0
for record in dataset:
    record_id+=1
    text=record['text']
    url=record['url']
    #urlフィルター
    is_rejected=url_filter.u_filter(url)
    if is_rejected==True:
        #print("rejected")
        continue
    #テキストクリーン
    #text=rule_based_docment_checker.process_text(text)
    #text=hojichar_filter.hoji_filter(text)
    text=clean_text(text)
    if text=="":
        #print(record_id)
        continue

    #記事の判定
    #is_noise=annotator.predict(text)
    #if is_noise==1:
        #continue

    #ジャンルの判定
    #genre=clf.predict(text)
    cnt+=1
    """
    d={
        "id":record_id,
        #"cat":genre,
        "text":text,
    }
    file_name=f"{corpus_dir}/{cnt//n_split}.txt"
    with open(file_name, "a") as f:
        f.write(str(d)+"\n")
    """
    if cnt>10000:
        #print(record_id)
        break

print(cnt/record_id)

0.13533519175079164


In [9]:
#圧縮率
cnt/record_id

0.19238095238095237

In [76]:
cnt/record_id*100

24.337349397590362

In [ ]:
#url_filterは1000に2つくらい

In [15]:
n_split=10**2
import os

corpus_dir="corpus/test"
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

cnt=0
record_id=-1
for record in dataset:
    record_id+=1
    if record_id<4:
        continue
    text=record['text']
    print(text)
    #テキストクリーン
    text=clean_text(text)
    if text=="":
        print("no")
        continue
    print("ok")
    cnt+=1

    d={
        "id":record_id,
        #"cat":genre,
        "text":text,
    }
    """
    file_name=f"{corpus_dir}/{cnt//n_split}.txt"
    with open(file_name, "a") as f:
        f.write(str(d)+"\n")
    """
    #print("フィルター後")
    #print(text)
    if cnt>100:
        break

【エロ動画】くりくり瞳のショートヘアの女の子にシコってもらってムハァーーーーーー！！(*ﾟ∀ﾟ)=3 アダルトMAX-無修正と無料動画-
エロ くりくり瞳のショートヘアの女の子にシコってもらってムハァーーーーーー *ﾟ∀ﾟ =3
TOP > エロ くりくり瞳のショートヘアの女の子にシコってもらってムハァーーーーーー *ﾟ∀ﾟ =3
投稿日:2015-08-02 01:00:16
カテゴリー エロ*ﾟ∀ﾟ=3
[1]«73298 73299 73300 73301 73302 73303 73304»[138086]
no
サーモス 真空断熱ポット コーヒーメーカー 【タイマーで前日予約が可能】 ECH-1...の
見た目の良さと、渋くなりにくい良さ, 2014/10/12
レビュー対象商品: サーモス 真空断熱ポット コーヒーメーカー 【タイマーで前日予約が可能】 ECH-1000 CS (ホーム&キッチン)
定年した父に、父の日のプレゼントとして購入しました。毎回コーヒーを水筒に入れ替えていたのを見て、こちらの方が効率がよいと思ったのが購入理由です。他の真空ポット系のコーヒーメーカーも見たのですが、THERMOSのものが一番良さそうでした。購入目的がおいしく淹れる目的ではないのですが一応蒸らしの時間が設定されているのも面白いです。タイマー機能も使っていないですが面白いです。ポットなので、ゴムパッキンに茶しぶ的なものがつくので、こま目に洗うことは必要です。ポットが冷たい時には中のコーヒーも冷まされてしまうのであたためた方が保温性能が上がると思います。上の蓋を開けると、コーヒー粉を淹れる部分があるのですが、縁の溝に飛び散った粉が貯まるようで、ふと見た時にカビていて衝撃を受けました。取外しもできない部分で、さらに溝が細く深いので爪楊枝を使って綺麗にしました。取り外しできる部分が本体には少ないので清潔に使いたいときには清掃が手間です。いろいろ書きましたが保温力のあるコーヒーメーカーだと一番いいと思います。
見た目の良さと、渋くなりにくい良さ
ベストレビュワーランキング: 16,272
ok
フィルター後
サーモス真空断熱ポットコーヒーメーカー【タイマーで前日予約が可能】ECH-1...の
見た目の良さと、渋くなりにくい良さ,2014/10/12
レビュー対象商品:サーモス

In [36]:
import re

def remove_if_dominated_by_pattern(text, pattern, threshold=0.4):
    # パターンに合致する部分を全て検索
    matches = re.findall(pattern, text)
    matched_length = sum(len(match) for match in matches)

    # テキスト全体に対するパターンの割合を計算
    pattern_ratio = matched_length / len(text) if len(text) > 0 else 0

    # 指定された閾値を超えている場合、空のテキストを返す
    if pattern_ratio > threshold:
        return ""

    # それ以外の場合、元のテキストを返す
    return text

# テスト
sample_text = "っl、、、2024-04-10, another date 2024-04-11, and yet another date 2024-04-12."
cleaned_text = remove_if_dominated_by_pattern(sample_text, r'\d{4}-\d{2}-\d{2}', threshold=0.4)
print(cleaned_text)  # 空のテキストが出力される


In [101]:
import re

def remove_if_dominated_by_pattern(text, pattern, threshold=0.4):
    matches = re.findall(pattern, text)
    matched_length = sum(len(match) for match in matches)
    pattern_ratio = matched_length / len(text) if len(text) > 0 else 0

    if pattern_ratio > threshold:
        return ""
    return text

def remove_if_dominated_by_patterns(text, patterns, threshold=0.4):
    total_matched_length = 0
    for pattern in patterns:
        matches = re.findall(pattern, text)
        total_matched_length += sum(len(match) for match in matches)

    pattern_ratio = total_matched_length / len(text) if len(text) > 0 else 0
    if pattern_ratio > threshold:
        return ""

    return text


def pattern_filter(text):
    pattern_dic = {
    'date': r'\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4}|\d{4}/\d{2}/\d{2}',
    'time': r'\d{2}:\d{2}(?::\d{2})?',
    'URL': r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
    'Copyright': r'\(c\)|©',
    'Navigation menu': r'[\w\s]+>[^>]+',
    'HTML tag': r'<[^>]+>'
    }
    new_text=text
    for _pattern_name, pattern in pattern_dic.items():
        new_text = remove_if_dominated_by_pattern(new_text, pattern, threshold=0.4)
    new_text=remove_if_dominated_by_patterns(new_text, [pattern_dic['date'], pattern_dic['time']], threshold=0.4)
    return new_text


In [102]:
print(pattern_filter("2020/03/06 23:54"))

In [109]:
footer_list = []
# ファイルを読み込み、各行をリストに追加
with open("/data/satori_hdd4/takumi/steve_ssd2/matsuo_llm/JapaneseWarcParser/mc4s/src/cleaner/hoji_dict/header_footer_keywords_ja.txt", "r", encoding="utf-8") as file:
    for line in file:
        # 改行文字を除去してリストに追加
        footer_list.append(line.strip())

# リストの内容を確認
print(footer_list)

['カテゴリー', 'タグ', '投稿日', '投稿番号', 'パスワード', 'お名前', 'キーワード', '分類', '一覧に戻る', 'からのコメント', 'アーカイブ', '検索', 'トップページ', 'ホーム>', 'トップ', 'ホームページ', '診療時間', '診療予約', '医院HP', '初めての方へ', '前の記事', '次の記事', '前の商品', '次の商品', '【[\\u4E00-\\u9FD0]+】', '金額', '税込', '送料別', 'サイズ等', '数量', '関連商品', '価格', '商品ID', '寄附へ進む', '[ 専用ビューアについて ]', '« _ | Main | _ »', '<前ページ', '次ページ>', '記事一覧', '前のページへ戻る', '<TEST_STRING_OF_KEYWORD>', 'この記事へのトラックバック一覧', '無断転載を禁ず', 'クリック']
